**<center><h1>T-SNE Notebook</h1></center>**

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import FenicsATL as FATL
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE

from multiprocessing import Process


Loading BokehJS ...

# **I. DATA INITIALIZATION**

I.2 FENICS 02

In [6]:
irrad = [106,107,108,109,110,116,117,118,119,120,126,127,128,129,130,136,137,138,139,140,1021,1022,1023,1024,1025,101,102,103,104,105,111,112,114,115,121,122,123,124,125,131,132,133,134,135]
LTT_id = [29,30,31,32,33,34,35,36,37,38,39,40,1041,1042,1043,1044,1045,1046,1047]
All = irrad + LTT_id

In [7]:
FENICS2 = FATL.FenicsVers_filter(FATL.FenicsData_read('/AtlasDisk/home2/sisaid/data/FENICS2_data.json'),irrad)

FastFeaturesToDrop = [("NoiseHG","Mean"),("NoiseHG","Mu"),
                      ("NoiseHGIG1","Mean"),("NoiseHGIG1","Mu"),
                      ("NoiseHGIG2","Mean"),("NoiseHGIG2","Mu"),
                      ("NoiseHGIG6","Mean"),("NoiseHGIG6","Mu"),
                      ("NoiseLG","Mean"),("NoiseLG","Mu"),
                      ("NoiseLGIG1","Mean"),("NoiseLGIG1","Mu"),
                      ("NoiseLGIG2","Mean"),("NoiseLGIG2","Mu"),
                      ("NoiseLGIG6","Mean"),("NoiseLGIG6","Mu")]

# FENICS2 = FENICS2[FENICS2.Board.burnTime.notna()]

# temp_allindex = []
# for id in FENICS2.Board.id.unique() :
#     temp_index = FENICS2[FENICS2.Board.id == id][FENICS2.Board.channel == FENICS2[FENICS2.Board.id == id].Board.channel.value_counts().index[0]].index.tolist()
#     temp_allindex = temp_allindex + temp_index
    
# FENICS2 = FENICS2.loc[temp_allindex]
FENICS2 = FENICS2[(FENICS2.Board.StatusFast != -1) | (FENICS2.Board.StatusSlow != -1)]

Fast2 = FATL.FenicsTestTable(FENICS2,'fast')
Slow2 = FATL.FenicsTestTable(FENICS2,'slow')


# Slow2 = FATL.Normalizer(Slow2)
# Fast2 = FATL.Normalizer(Fast2)

for feature in FastFeaturesToDrop:
    Fast2 = Fast2.drop(feature, axis=1) 

slow2_index = Slow2.index

Slow2.dropna(subset=[('Gain0','Linearity','SlopeLin'),('Gain1','Linearity','SlopeLin'),('Gain3','Linearity','SlopeLin'),('Gain4','Linearity','SlopeLin'),('Gain5','Linearity','SlopeLin'),
                     ('Gain0','Linearity','ConstLin'),('Gain1','Linearity','ConstLin'),('Gain3','Linearity','ConstLin'),('Gain4','Linearity','ConstLin'),('Gain5','Linearity','ConstLin'),], inplace=True)
Slow2.dropna(inplace=True,axis=1)
Fast2.drop(index=slow2_index.difference(Slow2.index), inplace=True)
FENICS2.drop(index=slow2_index.difference(Slow2.index), inplace=True)



FastSlow2 = pd.concat([Slow2, Fast2], axis=1)

In [8]:
FEN2_batch01_ids = [29,30,31,32,33,34,35,36,37,38,39,40]
FEN2_batch02_ids = [1041,1042,1043,1044,1045,1046,1047]
NIEL_ids = [101,102,103,104,105,111,112,114,115,121,122,123,124,125,131,132,133,134,135]
TID_ids = [106,107,108,109,110,116,117,118,119,120,126,127,128,129,130,136,137,138,139,140,1021,1022,1023,1024,1025]

AllFen2 = list(set(FENICS2.Board.id[(FENICS2.Board.StatusFast==0 ) | (FENICS2.Board.StatusSlow==0)])
               .union(
                set(FEN2_batch01_ids),
                set(FEN2_batch02_ids),
                set(NIEL_ids),
                set(TID_ids))
                )

AllFen2LTT = list(set(FEN2_batch01_ids).union(set(FEN2_batch02_ids)))


In [9]:
main_dir = '/users/divers/atlas/sisaid/home2/'

# **III. FENICS 2**

III.1 FENICS 02 : FAST DATA

In [36]:
tuples = [('Fast', 'NonNormed'),('Fast', 'Normed'),('Slow', 'NonNormed'),('Slow', 'Normed'),('FastSlow', 'NonNormed'),('FastSlow', 'Normed')]
mux = pd.MultiIndex.from_tuples(tuples)
Labels2 = pd.DataFrame(index=FENICS2.index,columns=mux)

In [37]:
Labels2

(                         
       F         S         F     
       a         l         a     
       s         o         s     
       t         w         t     
       ,         ,         S     
                           l     
       N         N         o     
       o         o         w     
       n    r    n    r    ,     
       N    m    N    m          
       o    e    o    e    N     
       r    d    r    d    o     
       m    )    m    )    n    r
       e  NaN    e  NaN    N    m
       d  NaN    d  NaN    o    e
       )  NaN    )  NaN    r    d
     NaN  NaN  NaN  NaN    m    )
     NaN  NaN  NaN  NaN    e  NaN
     NaN  NaN  NaN  NaN    d  NaN
     NaN  NaN  NaN  NaN    )  NaN
0    NaN  NaN  NaN  NaN  NaN  NaN
1    NaN  NaN  NaN  NaN  NaN  NaN
2    NaN  NaN  NaN  NaN  NaN  NaN
3    NaN  NaN  NaN  NaN  NaN  NaN
4    NaN  NaN  NaN  NaN  NaN  NaN
..   ...  ...  ...  ...  ...  ...
659  NaN  NaN  NaN  NaN  NaN  NaN
660  NaN  NaN  NaN  NaN  NaN  NaN
661  NaN  NaN  NaN  NaN  NaN  NaN
662  NaN  NaN  NaN  NaN  NaN  NaN
663  NaN  NaN  NaN  NaN  NaN  NaN

[655 rows x 6 columns]

In [25]:
CanalStatus = {
    'StatusSlow' : FENICS2.Board.StatusSlow,
    'StatusFast' :  FENICS2.Board.StatusFast
}

Normalization = {
    'NonNormed' : False,
    'Normed' : True,
                 }


for norm in Normalization.keys() :
    tsne_param = TSNE()
    tsne_param.fit(Fast2)

    tsne_bokeh = FATL.FenBokehTSNE(Fast2,normalize=Normalization[norm],perplexity=tsne_param.perplexity,n_iter=tsne_param.n_iter,learning_rate=tsne_param.learning_rate)

    OutputPath = main_dir+'fen2_output/t-sne/fast/HTML/'

    FATL.checkFolderAt(OutputPath)

    custom_filter = [('Combo',dict(labels_filter=irrad,
                                radio_categories=[dict(name = 'All',indexes=irrad),
                                                    dict(name = 'TID',indexes=TID_ids),
                                                    dict(name = 'NIEL',indexes=NIEL_ids),
                                                    ]))]

    tsne= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,FENICS2.Board.version,FENICS2.Board.burnTime,FENICS2.Board.StatusFast,FENICS2.Board.StatusSlow,
                            colors=FENICS2.Board.burnTime ,labels=FENICS2.Board.id,
                            tooltips=[("card","FENICS0@version @labels"),("burnTime","@burnTime"),
                                        ("StatusFast","@StatusFast"),("StatusSlow","@StatusSlow")])

    tsne.plotter(plotType='histogram',filters=custom_filter,
                title=f't-SNE : FENICS02 FAST; normed {tsne_bokeh.normed}; t-SNE params : perplexity={tsne_param.perplexity},n_iter={tsne_param.n_iter},learning_rate={tsne_param.learning_rate}')

    # tsne.save_as_html(path=OutputPath,filename=f'FEN2_FAST_t-SNE_perplexity_{tsne_param.perplexity}_wburn_normed_{tsne_bokeh.normed}.html')

    epsilon = 5
    min_samples = 2
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    dbscan.fit(tsne_bokeh.embedding)

    Labels2[('Fast',norm)]= dbscan.labels_   

    Cluster_Filtred= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,
                                        colors=Labels2[('Fast',norm)] ,labels=FENICS2.Board.id)

    Cluster_Filtred.plotter(plotType='histogram',filters=custom_filter,title='t-SNE : FENICS02 FAST (DBSCAN)')


    for status in CanalStatus.keys() :
        Cluster_Filtred= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,FENICS2.Board.version,FENICS2.Board.burnTime,FENICS2.Board.StatusFast,FENICS2.Board.StatusSlow,
                                            colors=CanalStatus[status] ,labels=FENICS2.Board.id,
                                            tooltips=[("card","FENICS0@version @labels"),("burnTime","@burnTime"),
                                                        ("StatusFast","@StatusFast"),("StatusSlow","@StatusSlow")])

        Cluster_Filtred.plotter(plotType='histogram',filters=custom_filter,
                                title=f't-SNE : FENICS02 FAST (Fail : {status}); normed {tsne_bokeh.normed}; t-SNE params : perplexity={tsne_param.perplexity}')
        
        # Cluster_Filtred.save_as_html(path=OutputPath,filename=f'FEN2_FAST_t-SNE_perplexity_{tsne_param.perplexity}_wSStatusSlow_normed_{tsne_bokeh.normed}.html')


III.2 FENICS 02 : SLOW DATA

In [27]:
Slow2_filtred = pd.DataFrame()
Slow2_filtred = Slow2
removed_features_lienarity = ['DACforPed']
Slow2_filtred = Slow2_filtred.drop([('Gain0',   'NoisePhys'),('Gain1',  'NoisePhys'),
                                    ('Gain2',   'NoisePhys'),('Gain3',   'NoisePhys'),
                                    ('Gain4',   'NoisePhys'),('Gain5',  'NoisePhys')], axis=1) 

Slow2_filtred = Slow2_filtred.drop([('Gain0',   'NoisePosPed'),('Gain1',    'NoisePosPed'),
                                    ('Gain2',   'NoisePosPed'),('Gain3',   'NoisePosPed'),
                                    ('Gain4',   'NoisePosPed'),('Gain5',  'NoisePosPed')], axis=1) 

for feature in removed_features_lienarity :
    Slow2_filtred = Slow2_filtred.drop([('Gain0',   'Linearity',   feature),('Gain1',   'Linearity',    feature),
                                        ('Gain2',   'Linearity',    feature),('Gain3',   'Linearity',    feature),
                                        ('Gain4',   'Linearity',    feature),('Gain5',   'Linearity',    feature)], axis=1)

In [34]:
tsne_param = TSNE()
tsne_param.fit(Slow2_filtred)

for norm in Normalization.keys() :
    tsne_bokeh = FATL.FenBokehTSNE(Slow2_filtred,normalize=Normalization[norm],perplexity=tsne_param.perplexity,n_iter=tsne_param.n_iter,learning_rate=tsne_param.learning_rate)

    OutputPath = main_dir+'fen2_output/t-sne/slow/HTML/'

    FATL.checkFolderAt(OutputPath)

    tsne= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,FENICS2.Board.version,FENICS2.Board.burnTime,FENICS2.Board.StatusFast,FENICS2.Board.StatusSlow,
                            colors=FENICS2.Board.burnTime ,labels=FENICS2.Board.id,
                            tooltips=[("card","FENICS0@version @labels"),("burnTime","@burnTime"),
                                        ("StatusFast","@StatusFast"),("StatusSlow","@StatusSlow")])

    tsne.plotter(plotType='histogram',filters=custom_filter,
                title=f't-SNE : FENICS02 SLOW; normed {tsne_bokeh.normed}; t-SNE params : perplexity={tsne_param.perplexity},n_iter={tsne_param.n_iter},learning_rate={tsne_param.learning_rate}')

    # tsne.save_as_html(path=OutputPath,filename=f'FEN2_SLOW_t-SNE_perplexity_{tsne_param.perplexity}_wburn_normed_{tsne_bokeh.normed}.html')

    epsilon = 5
    min_samples = 2
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    dbscan.fit(tsne_bokeh.embedding)

    Labels2[('Slow',norm)]= dbscan.labels_   

    Cluster_Filtred= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,
                                        colors=Labels2[('Slow',norm)] ,labels=FENICS2.Board.id)

    Cluster_Filtred.plotter(plotType='histogram',filters=custom_filter,title='t-SNE : FENICS02 SLOW (DBSCAN)')
    
    
    
    for status in CanalStatus.keys() :
        Cluster_Filtred= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,FENICS2.Board.version,FENICS2.Board.burnTime,FENICS2.Board.StatusFast,FENICS2.Board.StatusSlow,
                                            colors=CanalStatus[status] ,labels=FENICS2.Board.id,
                                            tooltips=[("card","FENICS0@version @labels"),("burnTime","@burnTime"),
                                                        ("StatusFast","@StatusFast"),("StatusSlow","@StatusSlow")])

        Cluster_Filtred.plotter(plotType='histogram',filters=custom_filter,
                                title=f't-SNE : FENICS02 SLOW (Fail : {status}); normed {tsne_bokeh.normed}; t-SNE params : perplexity={tsne_param.perplexity}')

        # Cluster_Filtred.save_as_html(path=OutputPath,filename=f'FEN2_SLOW_t-SNE_perplexity_{tsne_param.perplexity}_wStatusSlow_normed_{tsne_bokeh.normed}.html')
        

III.3 FENICS 02 : FAST + SLOW DATA

In [31]:
FastSlow2_filtred = pd.concat([Slow2_filtred, Fast2], axis=1)

In [32]:
tsne_param = TSNE()
tsne_param.fit(FastSlow2_filtred)

for norm in Normalization.keys() :
    tsne_bokeh = FATL.FenBokehTSNE(FastSlow2_filtred,normalize=Normalization[norm],perplexity=20,n_iter=200000,learning_rate=200)

    OutputPath = main_dir+'fen2_output/t-sne/fastslow/HTML/'

    FATL.checkFolderAt(OutputPath)


    tsne= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,FENICS2.Board.version,FENICS2.Board.burnTime,FENICS2.Board.StatusFast,FENICS2.Board.StatusSlow,
                            colors=FENICS2.Board.burnTime ,labels=FENICS2.Board.id,
                            tooltips=[("card","FENICS0@version @labels"),("burnTime","@burnTime"),
                                        ("StatusFast","@StatusFast"),("StatusSlow","@StatusSlow")])

    tsne.plotter(plotType='histogram',filters=custom_filter,
                title=f't-SNE : FENICS02 FAST + SLOW; normed {tsne_bokeh.normed}; t-SNE params : perplexity={tsne_param.perplexity},n_iter={tsne_param.n_iter},learning_rate={tsne_param.learning_rate}')

    # tsne.save_as_html(path=OutputPath,filename=f'FEN2_FASTSLOW_t-SNE_perplexity_{tsne_param.perplexity}_wburn_normed_{tsne_bokeh.normed}.html')

    epsilon = 5
    min_samples = 2
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    dbscan.fit(tsne_bokeh.embedding)

    Labels2[('FastSlow',norm)]= dbscan.labels_   

    Cluster_Filtred= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,
                                        colors=Labels2[('FastSlow',norm)] ,labels=FENICS2.Board.id)

    Cluster_Filtred.plotter(plotType='histogram',filters=custom_filter,title='t-SNE : FENICS02 FastSlow (DBSCAN)')
    
    
    for status in CanalStatus.keys() :
        Cluster_Filtred= FATL.FenBokehGrapher(tsne_bokeh.tsne_x,tsne_bokeh.tsne_y,FENICS2.Board.version,FENICS2.Board.burnTime,FENICS2.Board.StatusFast,FENICS2.Board.StatusSlow,
                                            colors=CanalStatus[status] ,labels=FENICS2.Board.id,
                                            tooltips=[("card","FENICS0@version @labels"),("burnTime","@burnTime"),
                                                        ("StatusFast","@StatusFast"),("StatusSlow","@StatusSlow")])

        Cluster_Filtred.plotter(plotType='histogram',filters=custom_filter,
                                title=f't-SNE : FENICS02 Fast + SLOW (Fail : {status}); normed {tsne_bokeh.normed}; t-SNE params : perplexity={tsne_param.perplexity}')
        # Cluster_Filtred.save_as_html(path=OutputPath,filename=f'FEN2_FASTSLOW_t-SNE_perplexity_{tsne_param.perplexity}_wStatusSlow_normed_{tsne_bokeh.normed}.html')


# Clusters comparaison

In [12]:
F1022 = Fast2[FENICS2.Board.id ==1022].describe() ; F1023 = Fast2[FENICS2.Board.id ==1023].describe()
F1022.compare(F1023)

Gain                                   LinearHGcor              \
     GainRatioLC            GainRatioSC               SlopeAmp               
            self      other        self      other        self       other   
mean   39.769103  39.795701   39.789072  39.801301  150.550249  151.164437   
std     0.230604   0.253334    0.098662   0.100631    0.386111    0.574606   
min    39.629343  39.659033   39.517456  39.564752  149.393643  149.182844   
25%    39.699895  39.724063   39.740964  39.724573  150.542148  151.160134   
50%    39.724356  39.758475   39.762336  39.782791  150.674671  151.315453   
75%    39.746295  39.788627   39.869405  39.884531  150.757791  151.497699   
max    41.093625  41.265365   39.971384  39.965716  151.128465  151.909882   

                                                     ... NoiseLGIG1            \
       SlopeInteg              LowGoodLin            ...     StdDev             
             self        other       self     other  ...       self     other   
mean  7367.109300  7384.862092   0.179082  0.182489  ...   1.064832  1.116045   
std     45.386164    46.087244   0.085366  0.066657  ...   0.011290  0.021897   
min   7206.244332  7225.023664   0.048808  0.100487  ...   1.025890  1.059419   
25%   7368.110872  7389.727178   0.126456  0.125753  ...   1.059539  1.102899   
50%   7370.338157  7392.935844   0.152730  0.176497  ...   1.064215  1.113399   
75%   7386.526655  7404.444039   0.229183  0.227600  ...   1.073000  1.135550   
max   7412.968930  7428.174298   0.405739  0.355440  ...   1.082904  1.153145   

     NoiseLGIG2                               NoiseLGIG6                      \
          Sigma              StdDev                Sigma              StdDev   
           self     other      self     other       self     other      self   
mean   1.060083  1.094417  1.063812  1.118748   1.061256  1.094048  1.064576   
std    0.010793  0.011592  0.010207  0.021726   0.011349  0.012703  0.011630   
min    1.022692  1.057159  1.030773  1.059558   1.020153  1.055154  1.023139   
25%    1.054289  1.090851  1.058683  1.106111   1.056586  1.089208  1.059823   
50%    1.061584  1.094801  1.064233  1.115802   1.062223  1.093225  1.066679   
75%    1.066356  1.102655  1.069850  1.139009   1.068241  1.100315  1.071342   
max    1.074863  1.114738  1.078556  1.159260   1.074943  1.116611  1.077622   

                
                
         other  
mean  1.117844  
std   0.022580  
min   1.057364  
25%   1.106194  
50%   1.115372  
75%   1.133506  
max   1.158934  

[7 rows x 126 columns]

In [35]:
Labels2

,"(Fast, NonNormed)","(Fast, Normed)","(Slow, NonNormed)","(Slow, Normed)","(FastSlow, NonNormed)","(FastSlow, Normed)"
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
659,0,0,2,4,3,0
660,0,0,2,4,3,0
661,0,0,2,4,3,0
662,0,0,2,4,3,0
